# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 02:58:10.110000 2938222 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 02:58:10.110000 2938222 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 02:58:11] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=37457, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=52603

[2025-08-17 02:58:11] Using default HuggingFace chat template with detected content format: string


W0817 02:58:19.712000 2939214 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 02:58:19.712000 2939214 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 02:58:19.845000 2939215 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 02:58:19.845000 2939215 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 02:58:21] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-17 02:58:21] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=32.50 GB): 100%|██████████| 4/4 [00:09<00:00,  2.40s/it]
[2025-08-17 02:58:41] Capture draft cuda graph end. Time elapsed: 10.07 s. mem usage=0.76 GB. avail mem=31.95 GB.
[2025-08-17 02:58:41] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=31.95 GB


Capturing batches (bs=1 avail_mem=50.61 GB): 100%|██████████| 4/4 [00:00<00:00, 116.03it/s]
[2025-08-17 02:58:42] Capture draft extend cuda graph end. Time elapsed: 1.58 s. mem usage=-18.66 GB. avail mem=50.61 GB.
[2025-08-17 02:58:42] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=50.61 GB
[2025-08-17 02:58:42] INFO:     Started server process [2938222]
[2025-08-17 02:58:42] INFO:     Waiting for application startup.
[2025-08-17 02:58:42] INFO:     Application startup complete.
[2025-08-17 02:58:42] INFO:     Uvicorn running on http://127.0.0.1:37457 (Press CTRL+C to quit)


[2025-08-17 02:58:43] INFO:     127.0.0.1:55588 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 02:58:43] INFO:     127.0.0.1:55602 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 02:58:43] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 02:58:44] INFO:     127.0.0.1:55614 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 02:58:44] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 02:58:48] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 02:58:49] INFO:     127.0.0.1:58406 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 02:58:55.611000 2942560 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 02:58:55.611000 2942560 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 02:58:56] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38898, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1070817

[2025-08-17 02:58:57] Using default HuggingFace chat template with detected content format: string


W0817 02:59:04.184000 2943010 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 02:59:04.184000 2943010 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 02:59:04.281000 2943011 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 02:59:04.281000 2943011 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 02:59:05] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-17 02:59:05] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=36.25 GB): 100%|██████████| 4/4 [00:16<00:00,  4.09s/it]
[2025-08-17 03:00:13] Capture draft cuda graph end. Time elapsed: 17.15 s. mem usage=32.16 GB. avail mem=4.27 GB.
[2025-08-17 03:00:13] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=4.27 GB


Capturing batches (bs=1 avail_mem=4.02 GB): 100%|██████████| 4/4 [00:00<00:00, 42.07it/s]
[2025-08-17 03:00:14] Capture draft extend cuda graph end. Time elapsed: 1.02 s. mem usage=0.27 GB. avail mem=4.00 GB.
[2025-08-17 03:00:14] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=4.00 GB


[2025-08-17 03:00:14] INFO:     Started server process [2942560]
[2025-08-17 03:00:14] INFO:     Waiting for application startup.
[2025-08-17 03:00:14] INFO:     Application startup complete.
[2025-08-17 03:00:14] INFO:     Uvicorn running on http://127.0.0.1:38898 (Press CTRL+C to quit)
[2025-08-17 03:00:14] INFO:     127.0.0.1:54354 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 03:00:15] INFO:     127.0.0.1:60602 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 03:00:15] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 03:00:17] INFO:     127.0.0.1:60614 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 03:00:17] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 03:00:19] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 03:00:19] INFO:     127.0.0.1:60630 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 03:00:27.821000 2952432 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 03:00:27.821000 2952432 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 03:00:29] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38321, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-17 03:00:29] Using default HuggingFace chat template with detected content format: string


W0817 03:00:36.658000 2953073 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 03:00:36.658000 2953073 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 03:00:36.658000 2953072 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 03:00:36.658000 2953072 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 03:00:38] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=53.75 GB): 100%|██████████| 4/4 [00:00<00:00, 80.26it/s]
[2025-08-17 03:01:07] Capture draft extend cuda graph end. Time elapsed: 0.63 s. mem usage=0.22 GB. avail mem=53.75 GB.
[2025-08-17 03:01:07] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=53.75 GB


[2025-08-17 03:01:08] INFO:     Started server process [2952432]
[2025-08-17 03:01:08] INFO:     Waiting for application startup.
[2025-08-17 03:01:08] INFO:     Application startup complete.
[2025-08-17 03:01:08] INFO:     Uvicorn running on http://127.0.0.1:38321 (Press CTRL+C to quit)


[2025-08-17 03:01:08] INFO:     127.0.0.1:37180 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 03:01:09] INFO:     127.0.0.1:37186 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 03:01:09] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 03:01:09] INFO:     127.0.0.1:37196 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 03:01:09] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 03:01:13] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 03:01:13] INFO:     127.0.0.1:37208 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 03:01:21.144000 2956369 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 03:01:21.144000 2956369 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 03:01:22] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=39594, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-17 03:01:23] Using default HuggingFace chat template with detected content format: string


W0817 03:01:29.749000 2957154 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 03:01:29.749000 2957154 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 03:01:29.763000 2957153 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 03:01:29.763000 2957153 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 03:01:30] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=3 avail_mem=58.09 GB):   0%|          | 0/4 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=58.07 GB): 100%|██████████| 4/4 [00:00<00:00, 11.16it/s]
[2025-08-17 03:01:58] Capture draft extend cuda graph end. Time elapsed: 1.14 s. mem usage=0.13 GB. avail mem=58.07 GB.
[2025-08-17 03:01:58] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=58.07 GB


[2025-08-17 03:01:59] INFO:     Started server process [2956369]
[2025-08-17 03:01:59] INFO:     Waiting for application startup.
[2025-08-17 03:01:59] INFO:     Application startup complete.
[2025-08-17 03:01:59] INFO:     Uvicorn running on http://127.0.0.1:39594 (Press CTRL+C to quit)
[2025-08-17 03:01:59] INFO:     127.0.0.1:44524 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 03:02:00] INFO:     127.0.0.1:44538 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 03:02:00] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 03:02:00] INFO:     127.0.0.1:44552 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 03:02:00] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-17 03:02:04] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-17 03:02:04] INFO:     127.0.0.1:44560 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0817 03:02:11.071000 2960990 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 03:02:11.071000 2960990 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 03:02:12] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=37401, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=658301407, constrained_jso

[2025-08-17 03:02:13] Using default HuggingFace chat template with detected content format: string


W0817 03:02:19.518000 2961829 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 03:02:19.518000 2961829 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0817 03:02:19.561000 2961830 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0817 03:02:19.561000 2961830 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-17 03:02:21] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-17 03:02:21] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=41.38 GB): 100%|██████████| 4/4 [00:00<00:00, 55.97it/s]
[2025-08-17 03:02:35] Capture draft extend cuda graph end. Time elapsed: 0.56 s. mem usage=0.09 GB. avail mem=41.38 GB.
[2025-08-17 03:02:35] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=41.38 GB


[2025-08-17 03:02:36] INFO:     Started server process [2960990]
[2025-08-17 03:02:36] INFO:     Waiting for application startup.
[2025-08-17 03:02:36] INFO:     Application startup complete.
[2025-08-17 03:02:36] INFO:     Uvicorn running on http://0.0.0.0:37401 (Press CTRL+C to quit)


[2025-08-17 03:02:36] INFO:     127.0.0.1:60990 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-17 03:02:37] INFO:     127.0.0.1:32770 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-17 03:02:37] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 03:02:37] INFO:     127.0.0.1:32778 - "POST /generate HTTP/1.1" 200 OK
[2025-08-17 03:02:37] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-17 03:02:41] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-17 03:02:41] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, accept len: 1.79, cuda graph: True, gen throughput (token/s): 11.70, #queue-req: 0, 


[2025-08-17 03:02:42] Decode batch. #running-req: 1, #token: 163, token usage: 0.01, accept len: 1.77, cuda graph: True, gen throughput (token/s): 164.88, #queue-req: 0, 


[2025-08-17 03:02:42] Decode batch. #running-req: 1, #token: 230, token usage: 0.01, accept len: 1.68, cuda graph: True, gen throughput (token/s): 148.67, #queue-req: 0, 


[2025-08-17 03:02:43] Decode batch. #running-req: 1, #token: 302, token usage: 0.01, accept len: 1.80, cuda graph: True, gen throughput (token/s): 162.25, #queue-req: 0, 


[2025-08-17 03:02:43] Decode batch. #running-req: 1, #token: 372, token usage: 0.02, accept len: 1.75, cuda graph: True, gen throughput (token/s): 162.88, #queue-req: 0, 
[2025-08-17 03:02:43] INFO:     127.0.0.1:32784 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).